In [1]:
import os

from DBApp.Knowledge.workflow import KnowledgeExtractionWorkflow
from DBApp.Storage.verctorstorage.esModule import ES_DB_Control, ES_DB_search

esdbcontrol = ES_DB_Control()

Connected to Elasticsearch: {'name': 'b94020e29ad8', 'cluster_name': 'docker-cluster', 'cluster_uuid': '9KKN5ZsUTtyRl9hUzdXFOQ', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [2]:
pwd

'/qe/data/project/project_cen/project_cen/2025code/牛马系统全模态知识库'

In [ ]:
# 测试代码
if __name__ == "__main__":
    # 创建工作流
    workflow =  KnowledgeExtractionWorkflow()

    # 执行完整流程
    custom_up_info = {
        "jb_ID": "010101",
        "temp_info": {
            "file_name": "01010_移动xxx",
            "temp_base_path": "DBApp/ModelServer/PDF2MDServer/OutputTemp/",
        },
        "save_info": {
            "CustomID": "CENzhongpei",
            "DBName": "testTempDB01",
            "DocName": "移动xxx",
        },
    }

    chunks = workflow.process(custom_up_info)

    # 输出结果
    for chunk in chunks:
        print(f"Chunk ID: {chunk.metadata.id}")
        print(f"Content: {chunk.content[:50]}...")
        # print(f"Metadata: {chunk.metadata}")
        print("-" * 50)

Chunk ID: 5fbd4658-c1a3-4399-9bc5-51e9db121dd8
Content: # 中国移动超级 SIM 卡技术白皮书（2020 年）  

中国移动研究院  

# 前 言  
...
--------------------------------------------------
Chunk ID: 69f40a6a-3d60-4d8b-80d6-f7490111aef4
Content: 本白皮书旨在明确中国移动对于个人领域 SIM 卡的发展方向、架构设计及能力要求，希望能够为产业在规划...
--------------------------------------------------
Chunk ID: 91486a21-37fb-4872-b556-eb2ba7a17f16
Content: 3.2.7 密码算法性能. ..9  
3.3 应用基础能力层技术要求.. 11  
3.3.1 G...
--------------------------------------------------
Chunk ID: 187f4bda-dc8b-480c-9b3b-4bd537bbe26d
Content: 2020 年是 5G 加速发展的发力年，根据国家“加快 5G 网络等新型基础设施建设，积极丰富 5G...
--------------------------------------------------
Chunk ID: 25b06399-cfa5-44cb-9b59-cf59b673e4da
Content: 本次中国移动推出的超级 SIM 卡，首先在 4G 基础上针对 3GPP 5G 标准中定义的新功能和新...
--------------------------------------------------
Chunk ID: a527fe75-477f-49b1-bfbc-fce25fa203dd
Content: 一是 NFC 刷卡。NFC 技术具有交易速度快、安全性高的优势，支持双离线场景（刷卡交易双方的设备均...
--------------------------------------------------
Chunk ID: 36a61a23-b43e-4b3e-a72c-

In [3]:
count = esdbcontrol.es.count(index='newtest01')

print("Document count:", count)
esdbcontrol.es.indices.refresh(index='newtest01')

query = {
    "aggs": {
        "by_db_name": {
            "terms": {
                "field": "db_name.keyword",
                "size": 10
            }
        }
    }
}

result = esdbcontrol.es.search(index='newtest01', body=query)
print("Aggregation result:", result['aggregations'])

Document count: {'count': 1, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}
Aggregation result: {'by_db_name': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': []}}


In [6]:
esdbcontrol.create_index()  # 设置向量维度

esdbcontrol.UpChunks2DB(chunks)  # 假设 chunks 已定义

Uploaded chunk 1/1: (28, [])
Index newtest01 refreshed


In [7]:
# 测试原始方法
db_names = esdbcontrol.get_unique_db_names_and_counts()
print("DB names:", db_names)

Total documents in index: 29
Found 2 unique db_names
DB names: [{'db_name': 'testTempDB01', 'count': 28}, {'db_name': 'test_db', 'count': 1}]


Uploaded chunk 1/1: (28, [])
Index newtest01 refreshed


In [8]:
esdbcontrol.get_unique_db_names_and_counts()

# esdbcontrol.get_unique_titles('testTempDB01')

Total documents in index: 29
Found 2 unique db_names


[{'db_name': 'testTempDB01', 'count': 28}, {'db_name': 'test_db', 'count': 1}]

In [4]:
mapping = esdbcontrol.es.indices.get_mapping(index='newtest01')
print("Index mapping:", mapping)

Index mapping: {'newtest01': {'mappings': {'properties': {'db_name': {'type': 'keyword'}, 'embeddingvector': {'type': 'dense_vector', 'dims': 1024, 'index': True, 'similarity': 'cosine', 'index_options': {'type': 'int8_hnsw', 'm': 16, 'ef_construction': 100}}, 'file_name': {'type': 'keyword'}, 'offset_end': {'type': 'integer'}, 'offset_start': {'type': 'integer'}, 'pagecontent': {'type': 'text'}}}}}


In [6]:
# esdbcontrol.delete_documents_by_db_name(db_name)

In [3]:
# from your_module import ES_DB_search

es_search = ES_DB_search()
results = es_search.hybrid_search(db_name='testTempDB01', query_text='中国', n=5)


print("Search results:", results)

Attempting to connect to http://localhost:19200 for ES_DB_search
Connected to Elasticsearch for ES_DB_search: {'name': 'b94020e29ad8', 'cluster_name': 'docker-cluster', 'cluster_uuid': '9KKN5ZsUTtyRl9hUzdXFOQ', 'version': {'number': '8.17.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2b6a7fed44faa321997703718f07ee0420804b41', 'build_date': '2024-12-11T12:08:05.663969764Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}
[('本白皮书旨在明确中国移动对于个人领域 SIM 卡的发展方向、架构设计及能力要求，希望能够为产业在规划设计 SIM 卡相关技术、产品和解决方案时提供参考和指引。利用超级 SIM 卡空间开放能力，中国移动希望联合各行业合作伙伴探索高质量的 SIM 卡业务，提升 SIM卡价值，构建 SIM 卡业务生态体系，服务用户生活。  \n\n本白皮书的版权归中国移动所有，未经授权，任何单位或个人不得复制或拷贝本建议之部分或全部内容。  \n\n# 目 录  \n\n1. 概述.. 2  \n2. 技术架构.. .5  \n3. 技术要求.. .6  \n3.1 硬件层.. 6  \n3.1.1 芯片安全要求. .6  \n3.1.2 物理电气特性. ..6  \n3.1.3 终端与卡通信接口.. 6  \n3.1.4 擦写寿命要求. 7  \n3.2 COS 层技术要求.. 7  \n3.2.1 基础电信功能. 7  \n3.2

In [10]:
results

[]